<a href="https://colab.research.google.com/github/cc-ca/projet-impression-3d/blob/main/projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import os
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import random
import time

In [26]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
%cd "/content/gdrive/MyDrive/projet impression 3d"
!pwd

/content/gdrive/MyDrive/projet impression 3d
/content/gdrive/MyDrive/projet impression 3d


https://drive.google.com/drive/folders/1_MU7jTR0kTt8MFQYrUzWiWc6uM-BHG_9?usp=drive_link

#multi classes-----------------------------

In [28]:
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (255, 255))  # Assurez-vous que la taille correspond à celle utilisée lors de l'entraînement
    img = img / 255.0  # Normalisez les valeurs des pixels
    img = np.expand_dims(img, axis=0)  # Ajoutez une dimension pour représenter le lot (batch)

    return img

def predict_defect(model, image_path):
    preprocessed_img = load_and_preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)

    # Interprétez la prédiction (1 signifie "OK", 0 signifie "défaut")
    if prediction[0, 0] > 0.5:
        return "OK"
    else:
        return "Défaut"


In [29]:
def load_data_multi_class(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):

            label = folder.lower()  # Utilisez le nom du dossier comme label
            print(label)
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (255, 255))
                data.append(img)
                labels.append(label)

    data = np.array(data) / 255.0
    labels = np.array(labels)

    return data, labels

In [30]:
!pwd

/content/gdrive/MyDrive/projet impression 3d


In [31]:
dataset_path = 'dataset'
data, labels = load_data_multi_class(dataset_path)

blobs
ok
spaghetti
under exstrosion


In [32]:
# Définir un dictionnaire de mapping catégorie -> numéro
categorie_mapping = {
    "blobs": 0,
    "ok": 1,
    "spaghetti": 2,
    "under exstrosion": 3
}

# Utiliser NumPy pour convertir les catégories en numéros
labels = np.array([categorie_mapping[c] for c in labels])

In [33]:
print (len(labels))
print(labels[-1])

177
3


In [34]:
combined_data = list(zip(data,labels))

# Mélangez la liste combinée
random.shuffle(combined_data)

# Divisez à nouveau la liste combinée en deux listes distinctes
data,labels = zip(*combined_data)

labels = np.array(labels)
data = np.array(data)
print (len(labels))
print(labels[-1])

177
0


In [35]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(255, 255, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),  # Ajout de la couche Dropout
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),  # Ajout de la couche Dropout
    layers.Dense(255, activation='relu'),
    layers.Dropout(0.5),  # Ajout de la couche Dropout
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Ajout d'une autre couche Dropout
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),  # Ajout d'une autre couche Dropout
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),  # Ajout d'une autre couche Dropout
    layers.Dense(4, activation='softmax')  # Ajustez cela en fonction de votre nombre de classes
])



In [36]:
from keras.callbacks import ModelCheckpoint

# Définissez le callback de sauvegarde du modèle
checkpoint = ModelCheckpoint('meilleur_modele.h5', monitor='val_accuracy', save_best_only=True)

# Compilez le modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [44]:
# Entraînez le modèle avec des batches de taille 10 par epoch et sauvegardez le meilleur modèle
model.fit(data, labels, epochs=10, validation_split=0.20, batch_size=10, callbacks=[checkpoint])


Epoch 1/10
15/15 [==============================] - 20s 1s/step - loss: 0.9867 - accuracy: 0.4610 - val_loss: 0.9145 - val_accuracy: 0.6389
Epoch 2/10
15/15 [==============================] - 1s 52ms/step - loss: 0.9670 - accuracy: 0.4610 - val_loss: 0.8696 - val_accuracy: 0.6389
Epoch 3/10
15/15 [==============================] - 1s 49ms/step - loss: 0.9447 - accuracy: 0.5461 - val_loss: 0.8827 - val_accuracy: 0.6389
Epoch 4/10
15/15 [==============================] - 1s 49ms/step - loss: 0.9114 - accuracy: 0.5603 - val_loss: 0.9343 - val_accuracy: 0.6389
Epoch 5/10
15/15 [==============================] - 1s 45ms/step - loss: 0.8549 - accuracy: 0.5816 - val_loss: 0.8799 - val_accuracy: 0.6389
Epoch 6/10
15/15 [==============================] - 1s 47ms/step - loss: 0.7829 - accuracy: 0.6312 - val_loss: 0.9009 - val_accuracy: 0.6389
Epoch 7/10
15/15 [==============================] - 8s 546ms/step - loss: 0.7379 - accuracy: 0.6383 - val_loss: 0.9628 - val_accuracy: 0.6944
Epoch 8/10
15

In [47]:
def predict_defect_multi_class(model, image_path):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    print (model.predict(preprocessed_img))

    # Interprétez les prédictions
    class_labels = ['blobs', 'OK', 'spaghetti','under exstrosion']
    predicted_class = class_labels[np.argmax(predictions)]

    return predicted_class

In [51]:
resultf=""
nb_total=0
nb_positif=0
for filename in os.listdir("testok"):
    #print(filename)
    nb_total+=1

    result = predict_defect_multi_class(model, "testok/" + filename)
    if result=='OK':
        nb_positif+=1

    resultf= resultf + "prédiction pour"+ filename + " : "+ str(result) +"\n"


1/1 [==============================] - 0s 28ms/step
[[0.04092408 0.8419949  0.07013928 0.04694179]]
1/1 [==============================] - 0s 30ms/step
[[2.6658777e-08 9.9999797e-01 1.1258160e-06 9.0291974e-07]]
1/1 [==============================] - 0s 26ms/step
[[2.5385295e-06 9.9993885e-01 3.2546894e-05 2.6142414e-05]]
1/1 [==============================] - 0s 26ms/step
[[0.00219448 0.9876252  0.00573768 0.00444271]]
1/1 [==============================] - 0s 37ms/step
[[1.3026742e-04 9.9862087e-01 7.0512830e-04 5.4378557e-04]]
1/1 [==============================] - 0s 27ms/step
[[1.8389619e-07 9.9999094e-01 4.6520768e-06 4.2135525e-06]]
1/1 [==============================] - 0s 26ms/step
[[2.5969724e-10 1.0000000e+00 1.5736946e-08 1.5734395e-08]]
1/1 [==============================] - 0s 27ms/step
[[1.6251786e-04 9.9795866e-01 1.0949534e-03 7.8392058e-04]]
1/1 [==============================] - 0s 27ms/step
[[1.31947618e-05 9.99803007e-01 1.02910715e-04 8.09146295e-05]]
1/1 [=======

In [52]:
print (resultf)
print(str((nb_positif*100)/nb_total) + " % de précision")

prédiction pourok.jpg : OK
prédiction pourspaghetti.jpg : OK
prédiction pourok3.jpg : OK
prédiction pourok4.jpg : OK
prédiction pourok5.png : OK
prédiction pourWIN_20231010_15_02_24_Pro.jpg : OK
prédiction pourWIN_20231010_15_02_40_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_00_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_09_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_11_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_18_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_24_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_29_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_56_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_58_Pro.jpg : OK
prédiction pourWIN_20231010_15_04_02_Pro.jpg : OK
prédiction pourWIN_20231010_15_04_05_Pro.jpg : OK

100.0 % de précision


In [ ]:
def capture():
  # Ouvrir la webcam (la webcam par défaut a l'ID 0)
  cap = cv2.VideoCapture(0)

  # Vérifier si la webcam est ouverte correctement
  if not cap.isOpened():
      print("Erreur: Impossible d'ouvrir la webcam.")
      exit()

  # Capturer une image
  time.sleep(1)
  ret, frame = cap.read()

  # Sauvegarder l'image capturée
  if ret:
      cv2.imwrite("photo_capturee.jpg", frame)
      print("Photo capturée avec succès.")
  else:
      print("Erreur lors de la capture de la photo.")

  # Libérer la webcam
  cap.release()
  time.sleep(1)


  result = predict_defect_multi_class(model, 'photo_capturee.jpg')
  print(f"Prédiction pour la capture camera: {result}")